In [1]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 80)

In [2]:
df = pd.read_csv("lc_loan.csv", low_memory=False)
df_test = pd.read_csv('test.csv', low_memory = False)
usa = pd.read_csv("us-state-codes.csv")

df_old= df.copy()
df_test_old= df_test.copy()

In [3]:
for i in set(df.columns)-set(df_test.columns): 
    del df[i]

In [4]:
#Drop the columns with high nan values 
none_pct = df.apply(lambda x: sum(x.isnull()))/df.shape[0]
high_nan_cols = none_pct[none_pct > 0.5].index
df = df.drop(high_nan_cols, axis=1)
df_test = df_test.drop(high_nan_cols, axis=1)

In [5]:
df.shape, df_test.shape

((887379, 52), (759338, 52))

In [6]:
given_features = list(df.columns)
req_float_cols = ['int_rate','revol_util','loan_amnt','dti',
                 'total_pymnt','installment','annual_inc',
                 'revol_util','revol_bal','tot_coll_amt','tot_cur_bal']

req_cat_cols = ['term','grade','emp_length','home_ownership',
               'acc_now_delinq','inq_last_6mths','delinq_2yrs']

all_cols = req_float_cols + req_cat_cols

In [7]:
drop_cols = list(set(given_features) - set(all_cols))

In [8]:
df[req_float_cols].dtypes == df_test[req_float_cols].dtypes

int_rate         True
revol_util       True
loan_amnt       False
dti              True
total_pymnt      True
installment      True
annual_inc       True
revol_util       True
revol_bal        True
tot_coll_amt     True
tot_cur_bal      True
dtype: bool

In [9]:
df[req_float_cols] = df[req_float_cols].astype('float')
df_test[req_float_cols] = df_test[req_float_cols].astype('float')

df[req_cat_cols] = df[req_cat_cols].astype('object')
df_test[req_cat_cols] = df_test[req_cat_cols].astype('object')

In [31]:
all_cols

['int_rate',
 'revol_util',
 'loan_amnt',
 'dti',
 'total_pymnt',
 'installment',
 'annual_inc',
 'revol_util',
 'revol_bal',
 'tot_coll_amt',
 'tot_cur_bal',
 'term',
 'grade',
 'emp_length',
 'home_ownership',
 'acc_now_delinq',
 'inq_last_6mths',
 'delinq_2yrs']

In [10]:
# df= df_old.copy()
# df_test = df_test_old.copy()

In [11]:
df['term'] = np.where(df['term'] ==' 36 months', 36, 60 )
df_test['term'] = np.where(df_test['term'] =='36 months', 36, 60 )

In [12]:
df.home_ownership = np.where(df.home_ownership == 'NONE', 'OTHER',df.home_ownership)
df_test.home_ownership = np.where(df_test.home_ownership == 'NONE', 'OTHER',df_test.home_ownership)

In [13]:
df['emp_length'] = np.where(df['emp_length'] == '< 1 year', '<1 year', df['emp_length'] )
df['emp_length'] = df['emp_length'].str.split(" ").str.get(0)

df_test['emp_length'] = np.where(df_test['emp_length'] == '< 1 year', '<1 year', df_test['emp_length'] )
df_test['emp_length'] = df_test['emp_length'].str.split(" ").str.get(0)

df.emp_length = np.where(df.emp_length == '<1', '0',df.emp_length)
df_test.emp_length = np.where(df_test.emp_length == '<1', '0',df_test.emp_length)

df.emp_length = df.emp_length.fillna('0')
df_test.emp_length = df.emp_length.fillna('0')

In [14]:
for col in ['inq_last_6mths','delinq_2yrs','acc_now_delinq']:
    df[col] = df[col].astype('float')
    df_test[col] = df_test[col].astype('float')

    df[col] = np.where(df[col] > 1.0, 2, df[col])
    df_test[col] = np.where(df_test[col] > 1.0, 2, df_test[col])

In [15]:
print(all(df.columns == df_test.columns))

True


In [16]:
df[req_cat_cols].head(1)

,term,grade,emp_length,home_ownership,acc_now_delinq,inq_last_6mths,delinq_2yrs
0,36,B,10+,RENT,0.0,1.0,0.0


In [17]:
#One hot encoding 
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False, drop_first = True)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [18]:
# def run(df,df_test,lst):
#     df = dummy_df(df,lst)
#     df_test = dummy_df(df_test,lst)
    
#     return df,df_test
    
# df,df_test = run(df,df_test,['inq_last_6mths'])
# print(all(df.columns == df_test.columns))

In [19]:
df = dummy_df(df, req_cat_cols)

In [20]:
df_test = dummy_df(df_test, req_cat_cols)

In [21]:
print(all(df.columns == df_test.columns))

True


In [22]:
df = df.drop(columns = drop_cols)
df_test = df_test.drop(columns = drop_cols)

In [23]:
print(all(df.columns == df_test.columns))

True


In [32]:
all_cols

['int_rate',
 'revol_util',
 'loan_amnt',
 'dti',
 'total_pymnt',
 'installment',
 'annual_inc',
 'revol_util',
 'revol_bal',
 'tot_coll_amt',
 'tot_cur_bal',
 'term',
 'grade',
 'emp_length',
 'home_ownership',
 'acc_now_delinq',
 'inq_last_6mths',
 'delinq_2yrs']

In [33]:
req_cat_cols

['term',
 'grade',
 'emp_length',
 'home_ownership',
 'acc_now_delinq',
 'inq_last_6mths',
 'delinq_2yrs']

In [24]:
#Adjusting the remaining nan values 
none_pct = df.apply(lambda x: sum(x.isnull()))/df.shape[0] 
col_with_nans = none_pct[none_pct > 0].index

for i in col_with_nans:
    if df[i].dtype == 'O': #If it i an object 
        df[i].fillna('missing',inplace=True)
    else:
        df[i].fillna(0,inplace=True)
        
none_pct = df_test.apply(lambda x: sum(x.isnull()))/df_test.shape[0] 
col_with_nans = none_pct[none_pct > 0].index

for i in col_with_nans:
    if df_test[i].dtype == 'O': #If it i an object 
        df_test[i].fillna('missing',inplace=True)
    else:
        df_test[i].fillna(0,inplace=True)

In [25]:
print(all(df.columns == df_test.columns))

True


In [26]:
#Creating a new column 
df['charge_off'] = np.where((df_old.loan_status == 'Charged Off') |
                               (df_old.loan_status == 'Does not meet the credit policy. Status:Charged Off'), 1, 0 )

#Creating a new column 
df_test['charge_off'] = np.where((df_test_old.loan_status == 'Charged Off') |
                               (df_test_old.loan_status == 'Does not meet the credit policy. Status:Charged Off'), 1, 0 )

In [27]:
df.to_csv("train_processed.csv")
df_test.to_csv("test_processed.csv")

In [28]:
df1 = pd.read_csv("train_processed.csv", low_memory=False)
df_test1 = pd.read_csv('test_processed.csv', low_memory = False)

In [29]:
df1.columns

Index(['Unnamed: 0', 'loan_amnt', 'int_rate', 'installment', 'annual_inc',
       'dti', 'revol_bal', 'revol_util', 'total_pymnt', 'tot_coll_amt',
       'tot_cur_bal', 'term_60', 'grade_B', 'grade_C', 'grade_D', 'grade_E',
       'grade_F', 'grade_G', 'emp_length_1', 'emp_length_10+', 'emp_length_2',
       'emp_length_3', 'emp_length_4', 'emp_length_5', 'emp_length_6',
       'emp_length_7', 'emp_length_8', 'emp_length_9',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'acc_now_delinq_1.0', 'acc_now_delinq_2.0',
       'inq_last_6mths_1.0', 'inq_last_6mths_2.0', 'delinq_2yrs_1.0',
       'delinq_2yrs_2.0', 'charge_off'],
      dtype='object')

In [30]:
print(all(df1.columns == df_test1.columns))

True
